In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
train=pd.read_csv('f:/nettrainsparse.csv',usecols=[1,2,3])
test=pd.read_csv('f:/nettestsparse.csv',usecols=[1,2,3])

In [76]:
no_movies = 9318
no_users = 8141

In [77]:
import time

In [78]:
rated= [set() for i in range(no_movies) ]
dictionary={}

In [79]:
deviation=np.zeros(no_movies*no_movies).reshape(no_movies,no_movies)
cardinality=np.zeros(no_movies*no_movies).reshape(no_movies,no_movies)

In [80]:
for i, row in enumerate(train.itertuples(), 1):
    usr=int(row.userId)
    mov=int(row.itemId)
    rate=row.rating
    
    rated[mov].add(usr)
    dictionary[(usr,mov)]=rate

In [81]:
for i in range(no_movies):
    t1=time.time()
    
    for j in range(i+1,no_movies):
        tmprate = list(rated[i].intersection(rated[j]))
        
        uoutput=0
        c=0
        for uid in tmprate:
            uoutput=uoutput+(dictionary[(uid,j)]-dictionary[(uid,i)])
            c=c+1
            
        deviation[i][j]=uoutput
        deviation[j][i]=-uoutput
        cardinality[i][j]=c
        cardinality[j][i]=c

In [82]:
test.describe()

,userId,itemId,rating
count,281575.000000,281575.000000,281575.000000
mean,4054.529667,4719.633053,3.613119
std,2363.180964,2685.522626,1.089395
min,0.000000,0.000000,1.000000
25%,1964.000000,2412.000000,3.000000
50%,4012.000000,4721.000000,4.000000
75%,6147.000000,7104.000000,4.000000
max,8140.000000,9317.000000,5.000000


In [83]:
import math

In [84]:
number=test['userId'].count()

In [85]:
rmse_normal2=0
mae_normal2=0
err_gim=0
err_gpim=0
c_gim=0
c_gpim=0
tp=0
fp=0
tn=0
fn=0
for i, row in enumerate(test.itertuples(), 1):
    usr=int(row.userId)
    mov=int(row.itemId)
    actual=row.rating
    pred_normal2=0
    pred_weighted2=0
    sum_weighted2=0
    list_of_all_movies_rate_by_the_current_user=train[train['userId']==usr]
    length=list_of_all_movies_rate_by_the_current_user.shape[0]
    cnt=0
    for j,r2 in enumerate(list_of_all_movies_rate_by_the_current_user.itertuples(),1):
        curmov=r2.itemId
        currate=r2.rating
        if cardinality[curmov][mov]!=0:
            pred_normal2+=(currate+(deviation[curmov][mov]/ cardinality[curmov][mov]))
            cnt+=1
    if cnt!=0:
        predicted_normal2=(pred_normal2/cnt)
    else:
        predicted_normal2=0
    if predicted_normal2>5:
        predicted_normal2=5
    if predicted_normal2 <0:
        predicted_normal2=0
    
    if actual >=3:
        err_gim+=math.fabs(actual-predicted_normal2)
        c_gim+=1
    if predicted_normal2>=3:
        err_gpim+=math.fabs(actual-predicted_normal2)
        c_gpim+=1
    if actual>=3 and predicted_normal2>=3:
        tp+=1
    elif actual<3 and predicted_normal2>=3:
        fp+=1
    elif actual>=3 and predicted_normal2<3:
        fn+=1
    else:
        tn+=1
    rmse_normal2+=((actual-predicted_normal2)**2)
    mae_normal2+=abs(actual-predicted_normal2)
    

In [86]:
print(mae_normal2/number)

0.731016426108


In [87]:
print(math.sqrt(rmse_normal2/number))

0.9337738941198033


In [88]:
print(err_gim/c_gim)

0.6049760877257646


In [89]:
print(err_gpim/c_gpim)

0.7212283733310411


In [90]:
tp/(tp+fp)

0.8952788562629866

In [91]:
tp/(tp+fn)

0.9010522983094906